In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split, KNeighborsClassifier
from scipy.stats import randint
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

ImportError: cannot import name 'KNeighborsClassifier' from 'sklearn.model_selection' (/Users/student/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/__init__.py)

In [ ]:
df = pd.read_csv('./chatgpt1.csv')

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['Username'].value_counts()

In [ ]:
plt.figure(figsize=(15,6))
sns.countplot(data=df, x='Username', order=df["Username"].value_counts().index[:20])
plt.xticks(rotation=90)
plt.show()

##Cleaning Data

In [ ]:
# Text Preprocessing
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    
    return text 

df['tidy_text'] = np.vectorize(remove_pattern)(df['Text'], "@[\w]*: | *RT*")
df[["Text", "tidy_text"]].head(5)

# Removing http/https
cleaned_tweets = []

for index, row in df.iterrows():
    
    # Here we are filtering out all the words that contains link
    words_without_links = [word for word in row.tidy_text.split() if 'http' not in word]
    cleaned_tweets.append(' '.join(words_without_links))

df['tidy_text'] = cleaned_tweets
df[["Text", "tidy_text"]].head(10)

# removing empty text
df = df[df['tidy_text']!='']
df[["Text", "tidy_text"]].head(5)

# Removing punctuations, numbers and special characters
df['All_tidy_text'] = df['tidy_text'].str.replace("[^a-zA-Z# ]", "")

SENTIMENT ANALYSIS

In [ ]:
from wordcloud import WordCloud, STOPWORDS
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import TextBlob

In [ ]:
sia = SentimentIntensityAnalyzer()
def find_sentiment(post):
    try:
        if sia.polarity_scores(post)["compound"] > 0:
            return "Positive"
        elif sia.polarity_scores(post)["compound"] < 0:
            return "Negative"
        else:
            return "Neutral"  
    except:
        return "Neutral"

In [ ]:
def plot_sentiment(df, feature, title):
    counts = df[feature].value_counts()
    percent = counts/sum(counts)

    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 5))
    
    colors = ["green", "red", "blue"]
    counts.plot(kind='bar', ax=ax1, color=colors)
    percent.plot(kind='bar', ax=ax2, color=colors)
    ax1.set_ylabel(f'Counts : {title} sentiments', size=12)
    ax2.set_ylabel(f'Percentage : {title} sentiments', size=12)
    plt.suptitle(f"Sentiment analysis: {title}")
    plt.tight_layout()
    plt.show()

In [ ]:
df['text_sentiment'] = df['Text'].apply(lambda x: find_sentiment(x))
plot_sentiment(df, 'text_sentiment', 'Text')

In [ ]:
df = df[['Text', 'Language']]

In [ ]:
df = df.dropna()

In [ ]:
x_train = df['Text']
y_train = df['Language']

In [ ]:
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2)

In [ ]:
tfidf = TfidfVectorizer()

x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

APPLYING DECISION TREE IN PYTHON

In [ ]:
 # Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(x_train_tfidf, y_train)

In [ ]:
#Predict the response for test dataset
y_pred = clf.predict(x_test_tfidf)

In [ ]:
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred) * 100, 4), "%")

APPLYING RANDOM FOREST

In [ ]:
# Create Random Forest classifer object
rf = RandomForestClassifier()

# Train Random Forest Classifer
rf.fit(x_train_tfidf, y_train)

In [ ]:
#Predict the response for test dataset
y_pred = rf.predict(x_test_tfidf)

In [ ]:
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred) * 100, 4), "%")

APPLYING KNN

In [ ]:
# Create KNN classifer object
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=61) 

# Train KNN Classifer
knn.fit(x_train_tfidf, y_train)

In [ ]:
#Predict the response for test dataset
y_pred = knn.predict(x_test_tfidf)

In [ ]:
print("Accuracy:", round(metrics.accuracy_score(y_test, y_pred) * 100, 4), "%")